In [1]:
import wbgapi as wb
import pandas as pd
import statistics as stats

In [64]:
country_map = wb.economy.DataFrame().reset_index()

In [66]:
country_map = country_map[['id', 'name', 'incomeLevel']]

In [67]:
country_map

,id,name,incomeLevel
0,ABW,Aruba,HIC
1,AFE,Africa Eastern and Southern,
2,AFG,Afghanistan,LIC
3,AFW,Africa Western and Central,
4,AGO,Angola,LMC
...,...,...,...
261,XKX,Kosovo,UMC
262,YEM,"Yemen, Rep.",LIC
263,ZAF,South Africa,UMC
264,ZMB,Zambia,LMC


In [26]:
country_map.reset_index(inplace=True)

country_map.head()

,id,name,aggregate,longitude,latitude,region,adminregion,lendingType,incomeLevel,capitalCity
0,ABW,Aruba,False,-70.0167,12.51670,LCN,,LNX,HIC,Oranjestad
1,AFE,Africa Eastern and Southern,True,NaN,NaN,,,,,
2,AFG,Afghanistan,False,69.1761,34.52280,MEA,MNA,IDX,LIC,Kabul
3,AFW,Africa Western and Central,True,NaN,NaN,,,,,
4,AGO,Angola,False,13.2420,-8.81155,SSF,SSA,IBD,LMC,Luanda


In [27]:
country_map = country_map[['id','name','incomeLevel']]
country_map.head()

,id,name,incomeLevel
0,ABW,Aruba,HIC
1,AFE,Africa Eastern and Southern,
2,AFG,Afghanistan,LIC
3,AFW,Africa Western and Central,
4,AGO,Angola,LMC


In [ ]:
# lookup for the indicators available in the library
wb.series.info()

In [330]:
selected_indicators = ['NY.GDP.MKTP.CD', 'ST.INT.RCPT.CD', 'ST.INT.ARVL','SP.POP.TOTL']
selected_countries = ['USA', 'MEX', 'FRA', 'ITA', 'JPN', 'THA']

In [331]:
data = wb.data.DataFrame(
    selected_indicators,
    selected_countries,
    time=range(2000, 2021)
)

In [332]:
data

YR2000        YR2001        YR2002  \
economy series                                                     
FRA     NY.GDP.MKTP.CD  1.360959e+12  1.370377e+12  1.492428e+12   
        SP.POP.TOTL     6.091866e+07  6.136438e+07  6.181214e+07   
        ST.INT.ARVL              NaN           NaN           NaN   
        ST.INT.RCPT.CD  3.870600e+10  3.782600e+10  4.111300e+10   
ITA     NY.GDP.MKTP.CD  1.149661e+12  1.172041e+12  1.281746e+12   
        SP.POP.TOTL     5.694211e+07  5.697698e+07  5.708982e+07   
        ST.INT.ARVL     6.270200e+07  6.096000e+07  6.356100e+07   
        ST.INT.RCPT.CD  2.871200e+10  2.692400e+10  2.821300e+10   
JPN     NY.GDP.MKTP.CD  4.968359e+12  4.374712e+12  4.182846e+12   
        SP.POP.TOTL     1.268430e+08  1.271490e+08  1.274450e+08   
        ST.INT.ARVL     4.757000e+06  4.772000e+06  5.239000e+06   
        ST.INT.RCPT.CD  5.970000e+09  5.750000e+09  6.069000e+09   
MEX     NY.GDP.MKTP.CD  7.420613e+11  7.960646e+11  8.106661e+11   
        SP.POP.TOTL     9.862555e+07  1.000991e+08  1.015486e+08   
        ST.INT.ARVL     1.056730e+08  1.007180e+08  1.001530e+08   
        ST.INT.RCPT.CD  9.133000e+09  9.190000e+09  9.547000e+09   
THA     NY.GDP.MKTP.CD  1.263922e+11  1.202965e+11  1.343009e+11   
        SP.POP.TOTL     6.300782e+07  6.365114e+07  6.426644e+07   
        ST.INT.ARVL     9.579000e+06  1.013300e+07  1.087300e+07   
        ST.INT.RCPT.CD  9.935000e+09  9.378000e+09  1.038800e+10   
USA     NY.GDP.MKTP.CD  1.025095e+13  1.058193e+13  1.092911e+13   
        SP.POP.TOTL     2.821624e+08  2.849690e+08  2.876252e+08   
        ST.INT.ARVL     7.834300e+07  7.097500e+07  6.443400e+07   
        ST.INT.RCPT.CD  1.183160e+11  1.054540e+11  9.741500e+10   

                              YR2003        YR2004        YR2005  \
economy series                                                     
FRA     NY.GDP.MKTP.CD  1.835096e+12  2.109792e+12  2.192146e+12   
        SP.POP.TOTL     6.224986e+07  6.270759e+07  6.318085e+07   
        ST.INT.ARVL              NaN  1.902820e+08  1.858290e+08   
        ST.INT.RCPT.CD  4.657600e+10  5.306800e+10  5.212600e+10   
ITA     NY.GDP.MKTP.CD  1.582930e+12  1.812809e+12  1.864982e+12   
        SP.POP.TOTL     5.739918e+07  5.782818e+07  5.816668e+07   
        ST.INT.ARVL     6.302600e+07  5.848000e+07  5.923000e+07   
        ST.INT.RCPT.CD  3.262300e+10  3.789300e+10  3.836400e+10   
JPN     NY.GDP.MKTP.CD  4.519562e+12  4.893116e+12  4.831467e+12   
        SP.POP.TOTL     1.277180e+08  1.277610e+08  1.277730e+08   
        ST.INT.ARVL     5.212000e+06  6.138000e+06  6.728000e+06   
        ST.INT.RCPT.CD  1.147500e+10  1.434300e+10  1.555400e+10   
MEX     NY.GDP.MKTP.CD  7.655500e+11  8.194592e+11  9.175719e+11   
        SP.POP.TOTL     1.029785e+08  1.043941e+08  1.058115e+08   
        ST.INT.ARVL     9.233000e+07  9.925000e+07  1.031460e+08   
        ST.INT.RCPT.CD  1.005800e+10  1.160900e+10  1.280100e+10   
THA     NY.GDP.MKTP.CD  1.522806e+11  1.728957e+11  1.893184e+11   
        SP.POP.TOTL     6.486817e+07  6.545205e+07  6.601742e+07   
        ST.INT.ARVL     1.008200e+07  1.173700e+07  1.156700e+07   
        ST.INT.RCPT.CD  1.045600e+10  1.305400e+10  1.210300e+10   
USA     NY.GDP.MKTP.CD  1.145645e+13  1.221720e+13  1.303920e+13   
        SP.POP.TOTL     2.901079e+08  2.928053e+08  2.955166e+08   
        ST.INT.ARVL     6.208200e+07  6.760600e+07  7.148400e+07   
        ST.INT.RCPT.CD  9.481400e+10  1.072100e+11  1.166820e+11   

                              YR2006        YR2007        YR2008  \
economy series                                                     
FRA     NY.GDP.MKTP.CD  2.317862e+12  2.655817e+12  2.926803e+12   
        SP.POP.TOTL     6.362234e+07  6.401689e+07  6.437512e+07   
        ST.INT.ARVL     1.938820e+08  1.933190e+08  1.935710e+08   
        ST.INT.RCPT.CD  5.461500e+10  6.389000e+10  6.803400e+10   
ITA     NY.GDP.MKTP.CD  1.958564e+12  2.222524e+12  2.417508e+12   
        SP.POP.T

In [333]:
data.index

MultiIndex([('FRA', 'NY.GDP.MKTP.CD'),
            ('FRA',    'SP.POP.TOTL'),
            ('FRA',    'ST.INT.ARVL'),
            ('FRA', 'ST.INT.RCPT.CD'),
            ('ITA', 'NY.GDP.MKTP.CD'),
            ('ITA',    'SP.POP.TOTL'),
            ('ITA',    'ST.INT.ARVL'),
            ('ITA', 'ST.INT.RCPT.CD'),
            ('JPN', 'NY.GDP.MKTP.CD'),
            ('JPN',    'SP.POP.TOTL'),
            ('JPN',    'ST.INT.ARVL'),
            ('JPN', 'ST.INT.RCPT.CD'),
            ('MEX', 'NY.GDP.MKTP.CD'),
            ('MEX',    'SP.POP.TOTL'),
            ('MEX',    'ST.INT.ARVL'),
            ('MEX', 'ST.INT.RCPT.CD'),
            ('THA', 'NY.GDP.MKTP.CD'),
            ('THA',    'SP.POP.TOTL'),
            ('THA',    'ST.INT.ARVL'),
            ('THA', 'ST.INT.RCPT.CD'),
            ('USA', 'NY.GDP.MKTP.CD'),
            ('USA',    'SP.POP.TOTL'),
            ('USA',    'ST.INT.ARVL'),
            ('USA', 'ST.INT.RCPT.CD')],
           names=['economy', 'series'])

In [334]:
data.loc[[('FRA','ST.INT.ARVL')]].mean(axis=1).values[0]

np.float64(194400352.94117647)

In [335]:
data.loc[[('FRA','ST.INT.RCPT.CD')]].mean(axis=1).values[0]

np.float64(57693761904.7619)

In [336]:
type(data.loc[[('FRA','ST.INT.RCPT.CD')]])

pandas.core.frame.DataFrame

In [337]:
data.loc[[('FRA','ST.INT.ARVL')]]["YR2000"].isna()

economy  series     
FRA      ST.INT.ARVL    True
Name: YR2000, dtype: bool

In [338]:
def ratiofill(fill_df, ref_df):
    """
    fill missing values in fill_df using the ratio of ref_df
    The ratio is calculated as the mean of the values in ref_df
    for each year, and then applied to fill_df.
    """

    # find the ratio of the reference dataframe
    ratio = fill_df.mean(axis=1).values[0]/ref_df.mean(axis=1).values[0]

    # find and fill the NaN values
    for column in fill_df.columns:
        if fill_df[column].isna().values[0]:
            fill_df[column].values[0] = ref_df[column].values[0] * ratio

    return fill_df.values

In [339]:
ratiofill(data.loc[[('FRA','ST.INT.ARVL')]], data.loc[[('FRA','ST.INT.RCPT.CD')]])

array([[1.30420687e+08, 1.27455508e+08, 1.38531124e+08, 1.56938819e+08,
        1.90282000e+08, 1.85829000e+08, 1.93882000e+08, 1.93319000e+08,
        1.93571000e+08, 1.92369000e+08, 1.89826000e+08, 1.96595000e+08,
        1.97522000e+08, 2.04410000e+08, 2.06599000e+08, 2.03302000e+08,
        2.03042000e+08, 2.07274000e+08, 2.11998000e+08, 2.17877000e+08,
        1.17109000e+08]])

In [340]:
data.loc[[('FRA','ST.INT.ARVL')]].values

array([[        nan,         nan,         nan,         nan, 1.90282e+08,
        1.85829e+08, 1.93882e+08, 1.93319e+08, 1.93571e+08, 1.92369e+08,
        1.89826e+08, 1.96595e+08, 1.97522e+08, 2.04410e+08, 2.06599e+08,
        2.03302e+08, 2.03042e+08, 2.07274e+08, 2.11998e+08, 2.17877e+08,
        1.17109e+08]])

In [353]:
# fix the missing values in arrival and tourism receipt of all countries
for country in data.index.levels[0]:
    data.loc[[(country, 'ST.INT.ARVL')]] = ratiofill(
        data.loc[[(country, 'ST.INT.ARVL')]],
        data.loc[[(country, 'ST.INT.RCPT.CD')]]
    )

    data.loc[[(country, 'ST.INT.RCPT.CD')]] = ratiofill(
        data.loc[[(country, 'ST.INT.RCPT.CD')]],
        data.loc[[(country, 'ST.INT.ARVL')]]
    )

In [365]:
data.index.levels[0]

Index(['FRA', 'ITA', 'JPN', 'MEX', 'THA', 'USA'], dtype='object', name='economy')

In [354]:
data

YR2000        YR2001        YR2002  \
economy series                                                     
FRA     NY.GDP.MKTP.CD  1.360959e+12  1.370377e+12  1.492428e+12   
        SP.POP.TOTL     6.091866e+07  6.136438e+07  6.181214e+07   
        ST.INT.ARVL     1.304207e+08  1.274555e+08  1.385311e+08   
        ST.INT.RCPT.CD  3.870600e+10  3.782600e+10  4.111300e+10   
ITA     NY.GDP.MKTP.CD  1.149661e+12  1.172041e+12  1.281746e+12   
        SP.POP.TOTL     5.694211e+07  5.697698e+07  5.708982e+07   
        ST.INT.ARVL     6.270200e+07  6.096000e+07  6.356100e+07   
        ST.INT.RCPT.CD  2.871200e+10  2.692400e+10  2.821300e+10   
JPN     NY.GDP.MKTP.CD  4.968359e+12  4.374712e+12  4.182846e+12   
        SP.POP.TOTL     1.268430e+08  1.271490e+08  1.274450e+08   
        ST.INT.ARVL     4.757000e+06  4.772000e+06  5.239000e+06   
        ST.INT.RCPT.CD  5.970000e+09  5.750000e+09  6.069000e+09   
MEX     NY.GDP.MKTP.CD  7.420613e+11  7.960646e+11  8.106661e+11   
        SP.POP.TOTL     9.862555e+07  1.000991e+08  1.015486e+08   
        ST.INT.ARVL     1.056730e+08  1.007180e+08  1.001530e+08   
        ST.INT.RCPT.CD  9.133000e+09  9.190000e+09  9.547000e+09   
THA     NY.GDP.MKTP.CD  1.263922e+11  1.202965e+11  1.343009e+11   
        SP.POP.TOTL     6.300782e+07  6.365114e+07  6.426644e+07   
        ST.INT.ARVL     9.579000e+06  1.013300e+07  1.087300e+07   
        ST.INT.RCPT.CD  9.935000e+09  9.378000e+09  1.038800e+10   
USA     NY.GDP.MKTP.CD  1.025095e+13  1.058193e+13  1.092911e+13   
        SP.POP.TOTL     2.821624e+08  2.849690e+08  2.876252e+08   
        ST.INT.ARVL     7.834300e+07  7.097500e+07  6.443400e+07   
        ST.INT.RCPT.CD  1.183160e+11  1.054540e+11  9.741500e+10   

                              YR2003        YR2004        YR2005  \
economy series                                                     
FRA     NY.GDP.MKTP.CD  1.835096e+12  2.109792e+12  2.192146e+12   
        SP.POP.TOTL     6.224986e+07  6.270759e+07  6.318085e+07   
        ST.INT.ARVL     1.569388e+08  1.902820e+08  1.858290e+08   
        ST.INT.RCPT.CD  4.657600e+10  5.306800e+10  5.212600e+10   
ITA     NY.GDP.MKTP.CD  1.582930e+12  1.812809e+12  1.864982e+12   
        SP.POP.TOTL     5.739918e+07  5.782818e+07  5.816668e+07   
        ST.INT.ARVL     6.302600e+07  5.848000e+07  5.923000e+07   
        ST.INT.RCPT.CD  3.262300e+10  3.789300e+10  3.836400e+10   
JPN     NY.GDP.MKTP.CD  4.519562e+12  4.893116e+12  4.831467e+12   
        SP.POP.TOTL     1.277180e+08  1.277610e+08  1.277730e+08   
        ST.INT.ARVL     5.212000e+06  6.138000e+06  6.728000e+06   
        ST.INT.RCPT.CD  1.147500e+10  1.434300e+10  1.555400e+10   
MEX     NY.GDP.MKTP.CD  7.655500e+11  8.194592e+11  9.175719e+11   
        SP.POP.TOTL     1.029785e+08  1.043941e+08  1.058115e+08   
        ST.INT.ARVL     9.233000e+07  9.925000e+07  1.031460e+08   
        ST.INT.RCPT.CD  1.005800e+10  1.160900e+10  1.280100e+10   
THA     NY.GDP.MKTP.CD  1.522806e+11  1.728957e+11  1.893184e+11   
        SP.POP.TOTL     6.486817e+07  6.545205e+07  6.601742e+07   
        ST.INT.ARVL     1.008200e+07  1.173700e+07  1.156700e+07   
        ST.INT.RCPT.CD  1.045600e+10  1.305400e+10  1.210300e+10   
USA     NY.GDP.MKTP.CD  1.145645e+13  1.221720e+13  1.303920e+13   
        SP.POP.TOTL     2.901079e+08  2.928053e+08  2.955166e+08   
        ST.INT.ARVL     6.208200e+07  6.760600e+07  7.148400e+07   
        ST.INT.RCPT.CD  9.481400e+10  1.072100e+11  1.166820e+11   

                              YR2006        YR2007        YR2008  \
economy series                                                     
FRA     NY.GDP.MKTP.CD  2.317862e+12  2.655817e+12  2.926803e+12   
        SP.POP.TOTL     6.362234e+07  6.401689e+07  6.437512e+07   
        ST.INT.ARVL     1.938820e+08  1.933190e+08  1.935710e+08   
        ST.INT.RCPT.CD  5.461500e+10  6.389000e+10  6.803400e+10   
ITA     NY.GDP.MKTP.CD  1.958564e+12  2.222524e+12  2.417508e+12   
        SP.POP.T

In [342]:
data

YR2000        YR2001        YR2002  \
economy series                                                     
FRA     NY.GDP.MKTP.CD  1.360959e+12  1.370377e+12  1.492428e+12   
        SP.POP.TOTL     6.091866e+07  6.136438e+07  6.181214e+07   
        ST.INT.ARVL              NaN           NaN           NaN   
        ST.INT.RCPT.CD  3.870600e+10  3.782600e+10  4.111300e+10   
ITA     NY.GDP.MKTP.CD  1.149661e+12  1.172041e+12  1.281746e+12   
        SP.POP.TOTL     5.694211e+07  5.697698e+07  5.708982e+07   
        ST.INT.ARVL     6.270200e+07  6.096000e+07  6.356100e+07   
        ST.INT.RCPT.CD  2.871200e+10  2.692400e+10  2.821300e+10   
JPN     NY.GDP.MKTP.CD  4.968359e+12  4.374712e+12  4.182846e+12   
        SP.POP.TOTL     1.268430e+08  1.271490e+08  1.274450e+08   
        ST.INT.ARVL     4.757000e+06  4.772000e+06  5.239000e+06   
        ST.INT.RCPT.CD  5.970000e+09  5.750000e+09  6.069000e+09   
MEX     NY.GDP.MKTP.CD  7.420613e+11  7.960646e+11  8.106661e+11   
        SP.POP.TOTL     9.862555e+07  1.000991e+08  1.015486e+08   
        ST.INT.ARVL     1.056730e+08  1.007180e+08  1.001530e+08   
        ST.INT.RCPT.CD  9.133000e+09  9.190000e+09  9.547000e+09   
THA     NY.GDP.MKTP.CD  1.263922e+11  1.202965e+11  1.343009e+11   
        SP.POP.TOTL     6.300782e+07  6.365114e+07  6.426644e+07   
        ST.INT.ARVL     9.579000e+06  1.013300e+07  1.087300e+07   
        ST.INT.RCPT.CD  9.935000e+09  9.378000e+09  1.038800e+10   
USA     NY.GDP.MKTP.CD  1.025095e+13  1.058193e+13  1.092911e+13   
        SP.POP.TOTL     2.821624e+08  2.849690e+08  2.876252e+08   
        ST.INT.ARVL     7.834300e+07  7.097500e+07  6.443400e+07   
        ST.INT.RCPT.CD  1.183160e+11  1.054540e+11  9.741500e+10   

                              YR2003        YR2004        YR2005  \
economy series                                                     
FRA     NY.GDP.MKTP.CD  1.835096e+12  2.109792e+12  2.192146e+12   
        SP.POP.TOTL     6.224986e+07  6.270759e+07  6.318085e+07   
        ST.INT.ARVL              NaN  1.902820e+08  1.858290e+08   
        ST.INT.RCPT.CD  4.657600e+10  5.306800e+10  5.212600e+10   
ITA     NY.GDP.MKTP.CD  1.582930e+12  1.812809e+12  1.864982e+12   
        SP.POP.TOTL     5.739918e+07  5.782818e+07  5.816668e+07   
        ST.INT.ARVL     6.302600e+07  5.848000e+07  5.923000e+07   
        ST.INT.RCPT.CD  3.262300e+10  3.789300e+10  3.836400e+10   
JPN     NY.GDP.MKTP.CD  4.519562e+12  4.893116e+12  4.831467e+12   
        SP.POP.TOTL     1.277180e+08  1.277610e+08  1.277730e+08   
        ST.INT.ARVL     5.212000e+06  6.138000e+06  6.728000e+06   
        ST.INT.RCPT.CD  1.147500e+10  1.434300e+10  1.555400e+10   
MEX     NY.GDP.MKTP.CD  7.655500e+11  8.194592e+11  9.175719e+11   
        SP.POP.TOTL     1.029785e+08  1.043941e+08  1.058115e+08   
        ST.INT.ARVL     9.233000e+07  9.925000e+07  1.031460e+08   
        ST.INT.RCPT.CD  1.005800e+10  1.160900e+10  1.280100e+10   
THA     NY.GDP.MKTP.CD  1.522806e+11  1.728957e+11  1.893184e+11   
        SP.POP.TOTL     6.486817e+07  6.545205e+07  6.601742e+07   
        ST.INT.ARVL     1.008200e+07  1.173700e+07  1.156700e+07   
        ST.INT.RCPT.CD  1.045600e+10  1.305400e+10  1.210300e+10   
USA     NY.GDP.MKTP.CD  1.145645e+13  1.221720e+13  1.303920e+13   
        SP.POP.TOTL     2.901079e+08  2.928053e+08  2.955166e+08   
        ST.INT.ARVL     6.208200e+07  6.760600e+07  7.148400e+07   
        ST.INT.RCPT.CD  9.481400e+10  1.072100e+11  1.166820e+11   

                              YR2006        YR2007        YR2008  \
economy series                                                     
FRA     NY.GDP.MKTP.CD  2.317862e+12  2.655817e+12  2.926803e+12   
        SP.POP.TOTL     6.362234e+07  6.401689e+07  6.437512e+07   
        ST.INT.ARVL     1.938820e+08  1.933190e+08  1.935710e+08   
        ST.INT.RCPT.CD  5.461500e+10  6.389000e+10  6.803400e+10   
ITA     NY.GDP.MKTP.CD  1.958564e+12  2.222524e+12  2.417508e+12   
        SP.POP.T

In [343]:
# # join the country with income levels
# data = data.merge(country_map, left_on='economy', right_on='id')
# data[data['economy']=='FRA']

In [3]:
data_long = data.melt(id_vars=['economy','series'],
                     var_name='year',
                     value_name='value')

data_long

,economy,series,year,value
0,FRA,NY.GDP.MKTP.CD,YR2000,1.360959e+12
1,FRA,SP.POP.TOTL,YR2000,6.091866e+07
2,FRA,ST.INT.ARVL,YR2000,1.304207e+08
3,FRA,ST.INT.RCPT.CD,YR2000,3.870600e+10
4,ITA,NY.GDP.MKTP.CD,YR2000,1.149661e+12
...,...,...,...,...
499,THA,ST.INT.RCPT.CD,YR2020,1.536000e+10
500,USA,NY.GDP.MKTP.CD,YR2020,2.135410e+13
501,USA,SP.POP.TOTL,YR2020,3.315777e+08
502,USA,ST.INT.ARVL,YR2020,4.503700e+07


In [4]:
data_long['year'] = data_long['year'].str.replace('YR','').astype(int)
data_long.rename(columns={
    'economy': 'country',
    'series': 'indicator'
})

data_long

,economy,series,year,value
0,FRA,NY.GDP.MKTP.CD,2000,1.360959e+12
1,FRA,SP.POP.TOTL,2000,6.091866e+07
2,FRA,ST.INT.ARVL,2000,1.304207e+08
3,FRA,ST.INT.RCPT.CD,2000,3.870600e+10
4,ITA,NY.GDP.MKTP.CD,2000,1.149661e+12
...,...,...,...,...
499,THA,ST.INT.RCPT.CD,2020,1.536000e+10
500,USA,NY.GDP.MKTP.CD,2020,2.135410e+13
501,USA,SP.POP.TOTL,2020,3.315777e+08
502,USA,ST.INT.ARVL,2020,4.503700e+07


In [4]:
data = pd.read_csv('./data/data.csv')
data

,economy,series,YR2000,YR2001,YR2002,YR2003,YR2004,YR2005,YR2006,YR2007,...,YR2011,YR2012,YR2013,YR2014,YR2015,YR2016,YR2017,YR2018,YR2019,YR2020
0,FRA,NY.GDP.MKTP.CD,1.360959e+12,1.370377e+12,1.492428e+12,1.835096e+12,2.109792e+12,2.192146e+12,2.317862e+12,2.655817e+12,...,2.870409e+12,2.683007e+12,2.816078e+12,2.861236e+12,2.442483e+12,2.470408e+12,2.588868e+12,2.781576e+12,2.722794e+12,2.647926e+12
1,FRA,SP.POP.TOTL,6.091866e+07,6.136438e+07,6.181214e+07,6.224986e+07,6.270759e+07,6.318085e+07,6.362234e+07,6.401689e+07,...,6.534083e+07,6.565766e+07,6.599793e+07,6.631207e+07,6.654827e+07,6.672410e+07,6.691802e+07,6.715835e+07,6.738206e+07,6.760111e+07
2,FRA,ST.INT.ARVL,1.304207e+08,1.274555e+08,1.385311e+08,1.569388e+08,1.902820e+08,1.858290e+08,1.938820e+08,1.933190e+08,...,1.965950e+08,1.975220e+08,2.044100e+08,2.065990e+08,2.033020e+08,2.030420e+08,2.072740e+08,2.119980e+08,2.178770e+08,1.171090e+08
3,FRA,ST.INT.RCPT.CD,3.870600e+10,3.782600e+10,4.111300e+10,4.657600e+10,5.306800e+10,5.212600e+10,5.461500e+10,6.389000e+10,...,6.613300e+10,6.397500e+10,6.605400e+10,6.740200e+10,6.644100e+10,6.355700e+10,6.771700e+10,7.251800e+10,7.077600e+10,3.595800e+10
4,ITA,NY.GDP.MKTP.CD,1.149661e+12,1.172041e+12,1.281746e+12,1.582930e+12,1.812809e+12,1.864982e+12,1.958564e+12,2.222524e+12,...,2.306974e+12,2.097929e+12,2.153226e+12,2.173256e+12,1.845428e+12,1.887111e+12,1.970721e+12,2.099435e+12,2.019607e+12,1.907481e+12
5,ITA,SP.POP.TOTL,5.694211e+07,5.697698e+07,5.708982e+07,5.739918e+07,5.782818e+07,5.816668e+07,5.839986e+07,5.875625e+07,...,6.002684e+07,6.019125e+07,6.031161e+07,6.032071e+07,6.022960e+07,6.011522e+07,6.000225e+07,6.014866e+07,5.972908e+07,5.943885e+07
6,ITA,ST.INT.ARVL,6.270200e+07,6.096000e+07,6.356100e+07,6.302600e+07,5.848000e+07,5.923000e+07,6.635300e+07,7.027100e+07,...,7.586600e+07,7.629300e+07,7.676200e+07,7.769400e+07,8.106800e+07,8.492500e+07,8.993100e+07,9.322860e+07,9.539900e+07,3.841900e+07
7,ITA,ST.INT.RCPT.CD,2.871200e+10,2.692400e+10,2.821300e+10,3.262300e+10,3.789300e+10,3.836400e+10,4.166600e+10,4.613400e+10,...,4.032903e+10,4.055601e+10,4.080533e+10,4.130076e+10,4.141500e+10,4.242300e+10,4.671900e+10,5.160200e+10,5.191000e+10,2.045900e+10
8,JPN,NY.GDP.MKTP.CD,4.968359e+12,4.374712e+12,4.182846e+12,4.519562e+12,4.893116e+12,4.831467e+12,4.601663e+12,4.579751e+12,...,6.233147e+12,6.272363e+12,5.212328e+12,4.896994e+12,4.444931e+12,5.003678e+12,4.930837e+12,5.040881e+12,5.117994e+12,5.054068e+12
9,JPN,SP.POP.TOTL,1.268430e+08,1.271490e+08,1.274450e+08,1.277180e+08,1.277610e+08,1.277730e+08,1.278540e+08,1.280010e+08,...,1.278330e+08,1.276290e+08,1.274450e+08,1.272760e+08,1.271410e+08,1.270760e+08,1.269720e+08,1.268110e+08,1.266330e+08,1.262610e+08


In [12]:
data = pd.read_csv('./test_data.csv')
data

,Unnamed: 0,country,indicator,year,value
0,0,ABW,NY.GDP.MKTP.CD,2000,1.873453e+09
1,1,ABW,SP.POP.TOTL,2000,9.058800e+04
2,2,ABW,ST.INT.ARVL,2000,1.211000e+06
3,3,ABW,ST.INT.RCPT.CD,2000,8.500000e+08
4,4,AFE,NY.GDP.MKTP.CD,2000,2.872017e+11
...,...,...,...,...,...
22339,22339,ZMB,ST.INT.RCPT.CD,2020,NaN
22340,22340,ZWE,NY.GDP.MKTP.CD,2020,2.686856e+10
22341,22341,ZWE,SP.POP.TOTL,2020,1.552689e+07
22342,22342,ZWE,ST.INT.ARVL,2020,6.390000e+05


In [13]:
data.drop(columns=['Unnamed: 0'], inplace=True)

In [41]:
pivot_data = data.pivot_table(
    index=['country', 'year'],
    columns='indicator',
    values='value'
).reset_index()

pivot_data

indicator,country,year,NY.GDP.MKTP.CD,SP.POP.TOTL,ST.INT.ARVL,ST.INT.RCPT.CD,arvl_per_gdp,rcpt_per_arvl,rcpt_per_gdp
0,ABW,2000,1.873453e+09,90588.0,1211000.0,8.500000e+08,0.000646,701.899257,0.453708
1,ABW,2001,1.896457e+09,91439.0,1178000.0,8.250000e+08,0.000621,700.339559,0.435022
2,ABW,2002,1.961844e+09,92074.0,1225000.0,8.350000e+08,0.000624,681.632653,0.425620
3,ABW,2003,2.044112e+09,93128.0,1184000.0,8.581000e+08,0.000579,724.746601,0.419791
4,ABW,2004,2.254831e+09,95138.0,1304000.0,1.057000e+09,0.000578,810.582822,0.468771
...,...,...,...,...,...,...,...,...,...
5560,ZWE,2016,2.054876e+10,14600294.0,2168000.0,1.940000e+08,0.000106,89.483395,0.009441
5561,ZWE,2017,5.107473e+10,14812482.0,2423000.0,1.580000e+08,0.000047,65.208419,0.003094
5562,ZWE,2018,3.415606e+10,15034452.0,2580000.0,1.910000e+08,0.000076,74.031008,0.005592
5563,ZWE,2019,2.571566e+10,15271368.0,2294000.0,2.850000e+08,0.000089,124.237140,0.011083


In [15]:
pivot_data['rcpt_per_arvl'] = pivot_data['ST.INT.RCPT.CD'] / pivot_data['ST.INT.ARVL']
pivot_data['rcpt_per_gdp'] = pivot_data['ST.INT.RCPT.CD'] / pivot_data['NY.GDP.MKTP.CD']
pivot_data['arvl_per_gdp'] = pivot_data['ST.INT.ARVL'] / pivot_data['NY.GDP.MKTP.CD']

In [49]:
pivot_data['arvl_per_pop'] = pivot_data['ST.INT.ARVL'] / pivot_data['SP.POP.TOTL']

In [16]:
pivot_data

indicator,country,year,NY.GDP.MKTP.CD,SP.POP.TOTL,ST.INT.ARVL,ST.INT.RCPT.CD,rcpt_per_arvl,rcpt_per_gdp,arvl_per_gdp
0,ABW,2000,1.873453e+09,90588.0,1211000.0,8.500000e+08,701.899257,0.453708,0.000646
1,ABW,2001,1.896457e+09,91439.0,1178000.0,8.250000e+08,700.339559,0.435022,0.000621
2,ABW,2002,1.961844e+09,92074.0,1225000.0,8.350000e+08,681.632653,0.425620,0.000624
3,ABW,2003,2.044112e+09,93128.0,1184000.0,8.581000e+08,724.746601,0.419791,0.000579
4,ABW,2004,2.254831e+09,95138.0,1304000.0,1.057000e+09,810.582822,0.468771,0.000578
...,...,...,...,...,...,...,...,...,...
5560,ZWE,2016,2.054876e+10,14600294.0,2168000.0,1.940000e+08,89.483395,0.009441,0.000106
5561,ZWE,2017,5.107473e+10,14812482.0,2423000.0,1.580000e+08,65.208419,0.003094,0.000047
5562,ZWE,2018,3.415606e+10,15034452.0,2580000.0,1.910000e+08,74.031008,0.005592,0.000076
5563,ZWE,2019,2.571566e+10,15271368.0,2294000.0,2.850000e+08,124.237140,0.011083,0.000089


In [17]:
data = pivot_data.melt(id_vars=['country', 'year'],
                       value_vars=['NY.GDP.MKTP.CD','SP.POP.TOTL',
                                   'ST.INT.RCPT.CD', 'ST.INT.ARVL',
                                   'rcpt_per_arvl', 'rcpt_per_gdp',
                                   'arvl_per_gdp'],
                       var_name='indicator',
                       value_name='value')
data

,country,year,indicator,value
0,ABW,2000,NY.GDP.MKTP.CD,1.873453e+09
1,ABW,2001,NY.GDP.MKTP.CD,1.896457e+09
2,ABW,2002,NY.GDP.MKTP.CD,1.961844e+09
3,ABW,2003,NY.GDP.MKTP.CD,2.044112e+09
4,ABW,2004,NY.GDP.MKTP.CD,2.254831e+09
...,...,...,...,...
38950,ZWE,2016,arvl_per_gdp,1.055052e-04
38951,ZWE,2017,arvl_per_gdp,4.744029e-05
38952,ZWE,2018,arvl_per_gdp,7.553565e-05
38953,ZWE,2019,arvl_per_gdp,8.920635e-05


In [19]:
data_pivot = data.pivot_table(
    index=['country', 'year'],
    columns='indicator',
    values='value'
).reset_index()

In [51]:
pivot_data.to_csv('./test_data.csv', index=False)

In [18]:
data.to_csv('test_data.csv', index=False)

In [40]:
pivot_data

In [42]:
pivot_data = pivot_data.merge(country_map, left_on='country', right_on='id', how='left')

In [50]:
#pivot_data.drop(columns=['id'],inplace=True,axis=1)
pivot_data

,country_code,year,NY.GDP.MKTP.CD,SP.POP.TOTL,ST.INT.ARVL,ST.INT.RCPT.CD,arvl_per_gdp,rcpt_per_arvl,rcpt_per_gdp,country_name,income_level,arvl_per_pop
0,ABW,2000,1.873453e+09,90588.0,1211000.0,8.500000e+08,0.000646,701.899257,0.453708,Aruba,HIC,13.368217
1,ABW,2001,1.896457e+09,91439.0,1178000.0,8.250000e+08,0.000621,700.339559,0.435022,Aruba,HIC,12.882906
2,ABW,2002,1.961844e+09,92074.0,1225000.0,8.350000e+08,0.000624,681.632653,0.425620,Aruba,HIC,13.304516
3,ABW,2003,2.044112e+09,93128.0,1184000.0,8.581000e+08,0.000579,724.746601,0.419791,Aruba,HIC,12.713684
4,ABW,2004,2.254831e+09,95138.0,1304000.0,1.057000e+09,0.000578,810.582822,0.468771,Aruba,HIC,13.706405
...,...,...,...,...,...,...,...,...,...,...,...,...
5560,ZWE,2016,2.054876e+10,14600294.0,2168000.0,1.940000e+08,0.000106,89.483395,0.009441,Zimbabwe,LMC,0.148490
5561,ZWE,2017,5.107473e+10,14812482.0,2423000.0,1.580000e+08,0.000047,65.208419,0.003094,Zimbabwe,LMC,0.163578
5562,ZWE,2018,3.415606e+10,15034452.0,2580000.0,1.910000e+08,0.000076,74.031008,0.005592,Zimbabwe,LMC,0.171606
5563,ZWE,2019,2.571566e+10,15271368.0,2294000.0,2.850000e+08,0.000089,124.237140,0.011083,Zimbabwe,LMC,0.150216


In [44]:
pivot_data.drop(columns=['id'], inplace=True, axis=1)

In [46]:
pivot_data.rename(columns={
    'country': 'country_code',
    'name': 'country_name',
    'incomeLevel': 'income_level'
}, inplace=True)

In [47]:
pivot_data

,country_code,year,NY.GDP.MKTP.CD,SP.POP.TOTL,ST.INT.ARVL,ST.INT.RCPT.CD,arvl_per_gdp,rcpt_per_arvl,rcpt_per_gdp,country_name,income_level
0,ABW,2000,1.873453e+09,90588.0,1211000.0,8.500000e+08,0.000646,701.899257,0.453708,Aruba,HIC
1,ABW,2001,1.896457e+09,91439.0,1178000.0,8.250000e+08,0.000621,700.339559,0.435022,Aruba,HIC
2,ABW,2002,1.961844e+09,92074.0,1225000.0,8.350000e+08,0.000624,681.632653,0.425620,Aruba,HIC
3,ABW,2003,2.044112e+09,93128.0,1184000.0,8.581000e+08,0.000579,724.746601,0.419791,Aruba,HIC
4,ABW,2004,2.254831e+09,95138.0,1304000.0,1.057000e+09,0.000578,810.582822,0.468771,Aruba,HIC
...,...,...,...,...,...,...,...,...,...,...,...
5560,ZWE,2016,2.054876e+10,14600294.0,2168000.0,1.940000e+08,0.000106,89.483395,0.009441,Zimbabwe,LMC
5561,ZWE,2017,5.107473e+10,14812482.0,2423000.0,1.580000e+08,0.000047,65.208419,0.003094,Zimbabwe,LMC
5562,ZWE,2018,3.415606e+10,15034452.0,2580000.0,1.910000e+08,0.000076,74.031008,0.005592,Zimbabwe,LMC
5563,ZWE,2019,2.571566e+10,15271368.0,2294000.0,2.850000e+08,0.000089,124.237140,0.011083,Zimbabwe,LMC
